# Assignement (segmenting and clustering)

In [4]:
import pandas as pd

## Get dataframe from wikipedia

In [5]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [6]:
# import data as dataframe with pandas and select the first table (index 0)
df_toronto = pd.read_html(link)[0]
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Clean dataframe

In [7]:
# rename postalcode column
df_toronto.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [8]:
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
#drop cells with "not-assined" borough
df_toronto = df_toronto[df_toronto['Borough']!='Not assigned']
df_toronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Format dataframe

In [10]:
#combine rows with same postal code and borough
df_toronto_merged = df_toronto.groupby(['PostalCode', 'Borough'], sort=False)

#merge neighbourhood names in same row
df_toronto_merged = df_toronto_merged['Neighbourhood'].apply(','.join).reset_index()

In [11]:
df_toronto_merged.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [12]:
#check for not assigned neighbourhood
df_toronto_merged[df_toronto_merged.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood
5,M9A,Queen's Park,Not assigned


In [13]:
#get indexes for not assigned neighbourhood
i_not_assigned = df_toronto_merged[df_toronto_merged.Neighbourhood == 'Not assigned'].index

In [14]:
#assign name of borough to not assigned neighbourhood
df_toronto_merged.loc[i_not_assigned, 'Neighbourhood'] = df_toronto_merged['Borough']

In [15]:
#checking all neighbourhood are assigned
df_toronto_merged[df_toronto_merged.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood


In [16]:
df_toronto_merged.shape

(103, 3)

## Get location data

We will use the Geocoder Python package (https://geocoder.readthedocs.io/index.html)

### update: the geocoder didn't work (keep returning None), so location extracted from csv file below

In [2]:
#installing the package 
!pip install geocoder

    Running setup.py install for future: started
    Running setup.py install for future: finished with status 'done'


In [32]:
import geocoder

In [17]:
df_toronto_merged.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [22]:
#getting list of all postal code
pc_list = df_toronto_merged['PostalCode'].to_list()

In [27]:
#initializing list of coordinates
lat_list = []
long_list = []

for postal_code in pc_list:
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        print(postal_code)
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    lat_list.append(lat_lng_coords[0])
    long_list.append(lat_lng_coords[1])

M3A
M3A
M3A
M3A
M3A
M3A
M3A


KeyboardInterrupt: 

## Get location data from csv

In [34]:
csv_link = 'http://cocl.us/Geospatial_data'

In [35]:
df_loc = pd.read_csv(csv_link)
df_loc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### First, we will rename the Postal Code column from the location dataframe to have the same name as the Toronto dataframe. 
### Then, we will merge the two dataframes based on the column 'PostalCode'

In [40]:
df_loc.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [45]:
df_toronto_final = pd.merge(df_toronto_merged, df_loc, on=['PostalCode'])

In [46]:
df_toronto_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
